In [1]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait   # para meter esperaras
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver import ActionChains # para hacer acciones con el ratón
import time
import random
import selenium.webdriver.common.keys as Keys
from selenium import webdriver
import time
#import warnings
#warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys

In [2]:
#Opciones de chrome
options = webdriver.ChromeOptions()
opciones = webdriver.ChromeOptions()
opciones.add_argument('--start-maximized')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_argument('--incognito')
#options.add_argument('--headless') # no abre la ventana de chrome

driver = webdriver.Chrome(options=options)   # abre una ventana de chrome
#driver.get('https://www.google.es')
driver.quit()

In [3]:
URL = 'https://www.google.es/search?q='
busqueda = 'mercadona'

In [4]:
driver = webdriver.Chrome(options = options)   # abre ventana

driver.get(URL+busqueda)          # entra en google y busca

time.sleep(3)                     # espera 8 secs

driver.find_element(By.ID, 'W0wltc').click()  #  rechaza las cookies

time.sleep(1)

link_mercadona = driver.find_element(By.CSS_SELECTOR,'#rso > div:nth-child(1) > div > div > div > div > div > div > div > div:nth-child(1) > div > span > a > h3')
link_mercadona.click()  #abre el link de mercadona

time.sleep(2)  

cookies_merca = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div > button:nth-child(2)')
cookies_merca.click() #acepta las cookies del mercadona

time.sleep(2)  

aviso_merca=driver.find_element(By.CSS_SELECTOR,'#root > div.ui-modal.ui-big-modal > div > div.ui-big-modal__actions > button.ui-button.ui-button--small.ui-button--primary.ui-button--positive')
aviso_merca.click() #acepta el aviso que salta


codigo_postal = driver.find_element(By.CSS_SELECTOR,'#root header form input') # Se localiza un campo de entrada mediante un selector CSS
codigo_postal.send_keys('28007')   
for num in '28007':
    codigo_postal.send_keys(num)
    time.sleep(random.uniform(0.3,0.80  )) #escribiendo el codigo postal para poder entrar 

codigo_postal.submit()

time.sleep(3)


categorias = driver.find_element(By.XPATH, '//*[@id="root"]/header/div[1]/nav/a[1]')
categorias.click() # Accede a las categorías
time.sleep(3)

# Genera XPaths para las categorías del 1 al 26
categorias_xpath = ['//*[@id="root"]/div[2]/div[1]/ul/li[{}]/div/button/span/label'.format(i) for i in range(1, 27)]

time.sleep(3)
# Listas para almacenar datos
nombres_lista = []
pesos_lista = []
precios_lista = []


for categoria in categorias_xpath: # itera a través de las categorías definidas en categorias_xpath
    try:
        # Haz clic en la categoría
        categoria = driver.find_element(By.XPATH, categoria)
        categoria.click()
        WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'product-cell__description-name')))

       
        while True:
            # extrae información de la página actual utilizando BeautifulSoup. Se obtienen los nombres, pesos y precios de los productos y se añaden a las respectivas listas.
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            nombres_lista.extend([producto.text for producto in soup.find_all('h4', class_='subhead1-r product-cell__description-name')])
            pesos_lista.extend([peso.text for peso in soup.find_all('div', class_='product-format product-format__size--cell')])
            precios_lista.extend([precio.text for precio in soup.find_all('div', class_='product-price')])

            # Haz scroll hasta el final de la página
            body = driver.find_element(By.TAG_NAME, 'body') #busca el elemento del cuerpo (<body>)  especifica que estamos buscando el elemento por su nombre de etiqueta, que en este caso es 'body'. El resultado se almacena en la variable body.
            body.send_keys(Keys.END)
            time.sleep(2)  # Ajustamos el tiempo

            # Comprueba si el botón para cargar más productos está presente si no esta sale del bucle while
            try:
                scroll_element = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[2]/div[1]/div/div/button')
                scroll_element.click()
                time.sleep(2)  # Ajustamos el tiempo
            except NoSuchElementException:
                break  # Sale del bucle si no hay más productos para cargar

    except Exception as e:
        print(f"No hay mas productos , paso al siguiente") # Se manejan excepciones dentro del bucle for
        
        continue




driver.quit()


No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas productos , paso al siguiente
No hay mas produ

se crea un diccionario con las listas que creamos 

In [5]:
dicc = {'productos': nombres_lista,'peso' : pesos_lista ,'precio':precios_lista}

crea el DF del diccionario 

In [6]:
df6= pd.DataFrame(dicc)

para guardar el df en google drive

In [ ]:
csv_path = r"G:\Mi unidad\SUPERMERCADO\df6.csv" # el df(numero).csv es el nombre que le damos al data frame para que lo guarde, usamos la ruta de acceso mas el nombre que el dimos
df6.to_csv(csv_path, index=False)

para abrir el df

In [ ]:
# Reemplaza 'ruta/al/archivo.csv' con la ruta real de tu archivo en Google Drive
ruta_archivo = 'G:\Mi unidad\SUPERMERCADO\df6.csv'

df6 = pd.read_csv(ruta_archivo)

In [ ]:
df6.drop_duplicates(inplace=True)
df6

,productos,peso,precio
0,"Aceite de oliva 0,4º Hacendado",Botella 1 L,"9,25 € /ud."
1,Aceite de oliva virgen extra Hacendado,Garrafa 3 L,"29,55 € /ud."
2,Aceite de oliva virgen extra Hacendado,Botella 1 L,"9,90 € /ud."
3,Aceite de oliva virgen extra Hacendado Gran Se...,Botella 750 ml,"8,10 € /ud."
4,Aceite de oliva 1º Hacendado,Botella 1 L,"9,25 € /ud."
...,...,...,...
2527,Gel fijador cabello extra fuerte Giorgi fijaci...,Bote 170 ml,"2,90 € /ud."
2528,Gel fijador cabello control total Giorgi fijac...,Bote 170 ml,"2,90 € /ud."
2529,Cera cabello Brillo Deliplus fijación 4,Bote 100 ml,"3,20 € /ud."
2530,Gel activador de rizos cabello Curl Perfect De...,Bote 200 ml,"2,95 € /ud."


abrimos todos los data frames creando una columa nueva que sera los productos y sus pesos juntos para poder unirlos y borramos duplicaados 

df= 29/12/2023

df2= 30/12/2023

df3= 02/01/2024

df4= 05/01/2024

In [ ]:
df = pd.read_csv('G:\Mi unidad\SUPERMERCADO\df.csv') 
df['productos2'] = df['productos'] +' ' + df['peso']  # Nueva columna llamada 'productos2' en el 'df'.Con la union de productos y peso separados por un espacio(porque habian nombres iguales pero son peso diferentes y que al borrar duplicados no me afecte)
df.drop_duplicates(subset=['productos2'], inplace=True) # Elimina las filas duplicadas basándose en la columna 'productos2', conservando solo la primera ocurrencia de cada combinación única.
df_new = pd.DataFrame(df['productos2']) # Se crea un nuevo DataFrame llamado 'df_new'


df2 = pd.read_csv('G:\Mi unidad\SUPERMERCADO\df2.csv')
df2['productos2'] = df2['productos'] +' ' + df2['peso']
df2.drop_duplicates(subset=['productos2'], inplace=True)
df_new2 = pd.DataFrame(df2['productos2'])


df3 = pd.read_csv('G:\Mi unidad\SUPERMERCADO\df3.csv')
df3['productos2'] = df3['productos'] +' ' + df3['peso']
df3.drop_duplicates(subset=['productos2'], inplace=True)
df_new3 = pd.DataFrame(df3['productos2'])


df4 = pd.read_csv('G:\Mi unidad\SUPERMERCADO\df4.csv')
df4['productos2'] = df4['productos'] +' ' + df4['peso']
df4.drop_duplicates(subset=['productos2'], inplace=True)
df_new4 = pd.DataFrame(df4['productos2'])



df6 = pd.read_csv('G:\Mi unidad\SUPERMERCADO\df6.csv')
df6['productos2'] = df6['productos'] +' ' + df6['peso']
df6.drop_duplicates(subset=['productos2'], inplace=True)
df_new6 = pd.DataFrame(df6['productos2'])

se crea una lista con todos los productos de todos los data frames hacemos un concat para unir los 4 df por si no hay prodcutos en alguno que los meta en el nuevo df y borrarmos duplicados

In [ ]:
dataframes = [df_new, df_new2, df_new3, df_new4,df_new6] # Se crea una lista que contiene cuatro DataFrames: df_new, df_new2, df_new3, y df_new4.
df_new5 = pd.concat(dataframes, ignore_index=True) # Crea un nuevo df_new5, que es la concatenación de los df en la lista. Los índices se reinician para que sean consecutivos.
df_new5.drop_duplicates(inplace = True) # Eliminar las filas duplicadas en el df_new5.

hacemos un merge para que me una todos los precios de todos los data frames con la columna de productos2 

In [ ]:
df_new5 = df_new5.merge(df[['productos2', 'precio']], on='productos2', how='left', suffixes=('_1', '')) #suffixes=('_1', '') para agregar sufijos a las columnas de df_new5 y df, en caso de que haya columnas con los mismos nombres en ambos DataFrames.
df_new5 = df_new5.merge(df2[['productos2', 'precio']], on='productos2', how='left', suffixes=('_2', ''))
df_new5 = df_new5.merge(df3[['productos2', 'precio']], on='productos2', how='left', suffixes=('_3', ''))
df_new5 = df_new5.merge(df4[['productos2', 'precio']], on='productos2', how='left', suffixes=('_4', ''))
df_new5 = df_new5.merge(df6[['productos2', 'precio']], on='productos2', how='left', suffixes=('_5', ''))
df_new5.reset_index(drop=True)

# (merge)  Esto realiza la operación de combinación (merge) entre el DataFrame df_new5 y el subconjunto seleccionado de df.
# (ON) Especifica la columna que se utilizará como clave de unión. En este caso, la columna 'productos2' se utiliza como clave para combinar los DataFrames.
# (how='left') Indica que se realizará una combinación izquierda. 
# (suffixes) Agrega sufijos a las columnas para distinguir las columnas 'precio'

,productos2,precio_2,precio_3,precio_4,precio_5,precio
0,"Aceite de oliva 0,4º Hacendado Botella 1 L","8,15 € /ud.","8,15 € /ud.","8,15 € /ud.","8,15 € /ud.","9,25 € /ud."
1,Aceite de oliva virgen extra Hacendado Garrafa...,"27,60 € /ud.","27,60 € /ud.","27,60 € /ud.","27,60 € /ud.","29,55 € /ud."
2,Aceite de oliva virgen extra Hacendado Botella...,"9,25 € /ud.","9,25 € /ud.","9,25 € /ud.","9,25 € /ud.","9,90 € /ud."
3,Aceite de oliva virgen extra Hacendado Gran Se...,"7,60 € /ud.","7,60 € /ud.","7,60 € /ud.","7,60 € /ud.","8,10 € /ud."
4,Aceite de oliva 1º Hacendado Garrafa 5 L,"40,50 € /ud.","40,50 € /ud.","40,50 € /ud.","40,50 € /ud.",NaN
...,...,...,...,...,...,...
5378,Champú Hydra hyaluronic Deliplus cabello de no...,NaN,NaN,NaN,NaN,"2,35 €2,05 € /ud."
5379,Champú anticaspa Citrus Fresh H&S cabello gras...,NaN,NaN,NaN,NaN,"5,30 € /ud."
5380,Champú anticaspa Menthol Fresh H&S Bote 400 ml,NaN,NaN,NaN,NaN,"5,30 € /ud."
5381,Champú y acondicionador anticaspa Classic 2 en...,NaN,NaN,NaN,NaN,"5,30 € /ud."


In [ ]:
columna = df_new5.pop('precio')
df_new5.insert(1, 'precio', columna)

In [ ]:
df_new5

,productos2,precio,precio_2,precio_3,precio_4,precio_5
0,"Aceite de oliva 0,4º Hacendado Botella 1 L","9,25 € /ud.","8,15 € /ud.","8,15 € /ud.","8,15 € /ud.","8,15 € /ud."
1,Aceite de oliva virgen extra Hacendado Garrafa...,"29,55 € /ud.","27,60 € /ud.","27,60 € /ud.","27,60 € /ud.","27,60 € /ud."
2,Aceite de oliva virgen extra Hacendado Botella...,"9,90 € /ud.","9,25 € /ud.","9,25 € /ud.","9,25 € /ud.","9,25 € /ud."
3,Aceite de oliva virgen extra Hacendado Gran Se...,"8,10 € /ud.","7,60 € /ud.","7,60 € /ud.","7,60 € /ud.","7,60 € /ud."
4,Aceite de oliva 1º Hacendado Garrafa 5 L,NaN,"40,50 € /ud.","40,50 € /ud.","40,50 € /ud.","40,50 € /ud."
...,...,...,...,...,...,...
5378,Champú Hydra hyaluronic Deliplus cabello de no...,"2,35 €2,05 € /ud.",NaN,NaN,NaN,NaN
5379,Champú anticaspa Citrus Fresh H&S cabello gras...,"5,30 € /ud.",NaN,NaN,NaN,NaN
5380,Champú anticaspa Menthol Fresh H&S Bote 400 ml,"5,30 € /ud.",NaN,NaN,NaN,NaN
5381,Champú y acondicionador anticaspa Classic 2 en...,"5,30 € /ud.",NaN,NaN,NaN,NaN
